# INTELIGENCIA ARTIFICIAL - EVALUACIÓN 1
Integrantes: Sebastian Racedo y Sebastian Valle

Primero importaremos las librerias que necesitarems para llevar a cabo la implementación de nuestro algoritmo.
 - Usaremos la libreria math para operaciones matematicas al momento de calcular la entropia
 - Usaremos la libreria pandas para poder manejar los datasets de una manera optima
 - Usaremos la libreria numpy para las operaciones con matrices

In [1]:
# Libraries Import
import math
import pandas as pd
import numpy as np

Explicar clase Node y como sera el proceso de creación de nodos

In [2]:
class Node:
    def __init__(self, value):
        self.children = []
        self.value = value
        
    def addNode(node):
        self.children.push(node)

La función SplitData recibe como entrada un archivo csv que contiene la informacion del dataset y se encarga de separarla y convertirla en un dataframe lo que la hace mas manejable.
* Input: RawData (Data cruda)
* Output: data convertida en un dataframe, X siendo una matriz que contiene todos los atributos del dataset y Y siendo un vector columna con el atributo a predecir

In [3]:
# Read and split data into training and test sets
def SplitData(Raw_Data):
    data = pd.read_csv(Raw_Data,sep = ',')
    [n,m] = data.shape
    headers = list(data.columns.values)
    X = data.iloc[:,0:m-1]
    y = data.iloc[:,m-1]
    return data, X, y

La función Entropy calcula la entropia de la data ingresada mediante la formula: 
$$ Entropy= -\sum_j p_j*log_2(p_j) $$
Where $p_j$ is the number of ocurrences by events divide by the total number of elements in the dataset

In [4]:
def Entropy(data):
    Size_data = len(data)
    if Size_data == 0:
        return 0
    num_classes = [0 for i in data.iloc[:,-1].dropna().value_counts()]
    num_classes = data.iloc[:,-1].dropna().value_counts().tolist()
    num_classes = [x/Size_data for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num*math.log(num,2)
    return ent*-1

## GainRatio DEBE aceptar valores numericos o NO numericos, Actualmente solo sirve para NO numericos

La función Gain_Ratio recibe como entrada los data a los cuales se le va a calcular la proporción de ganancia y los headers que son los nombres de cada atributo. 

Los datos pueden ser tanto nomiales como continuos. 
- Si los datos son nominales:
    - Se divide la data en atributos y por cada atributo se sacan sus valores, a estos valores se les calcula su entropia y se decide cual tiene una mayor proporción de ganancia.
- Si los datos son continuos
    - Algo

La función retorna un vector con la proporción de ganancia por atributo, junto con sus valores

In [5]:
def Gain_Ratio(data,headers):
    List_entropy = []
    List_Sum= []
    List_split = []
    List_values_Per_Attribute = [] 
    Global_Entropy = Entropy(data)
    for attribute in headers[0:len(headers)-1]:
        if (np.issubdtype(data[attribute].dtype, np.number) == False):
            entropy = []
            values = []
            Split = 0
            for value in data[attribute].value_counts().index:
                # Gain Calculation
                values.append(value)
                Neg_branch = data.loc[(data[attribute] == value) & (data[headers[len(headers)-1]] == 'No')]
                Pos_branch  = data.loc[(data[attribute] == value) & (data[headers[len(headers)-1]] == 'Yes')]
                Total_branch = [Neg_branch,Pos_branch]
                Total_branch = pd.concat(Total_branch)
                probability_num = (len(Total_branch)/len(data))
                entropy.append(probability_num*Entropy(Total_branch))
                Split += probability_num*math.log(probability_num,2)
            Split = Split*-1
            List_values_Per_Attribute.append(list(values))
            List_split.append(Split)
            Sum_per_branch = sum(entropy)
            List_entropy.append(list(entropy))
            List_Sum.append(Sum_per_branch)
        else:
            pass
            #print(data.sort_values(by = [attribute]))
    Gain = np.subtract(Global_Entropy,List_Sum)
    GainRatio = np.divide(Gain,List_split)
    return GainRatio, List_values_Per_Attribute

La función SplitAttributes recibe como entrada la data que se dividira junto con los headers de esta, este algoritmo funciona de forma recursiva. Primero se llamara la función Gain_Ratio la cual calculara las proporciones de ganancia, despues se escogere el atributo con la mayor proporción de ganancia y se escogeran sus valores. Despues estos seran pasados a la función GetSubs la cual se encargara de sacaran **los subconjuntos de acuerdo a los valores del atributo que tuvo la mayor proporción de ganancia**, y con estos se volvera a llamar a la función SplitAttributes.

In [21]:
def SplitAttributes(RemainingData, headers):
    headers = list(RemainingData.columns.values)
    split_values = []
    if(len(headers) == 1):
        return None
    else:
        print("Current headers " + str(headers))
        GainRatio, values = Gain_Ratio(RemainingData,headers)
        index = np.argmax(GainRatio)
        GainPos = GainRatio[index]
        Best_Attribute = headers[index]
        Best_Values = values[index]
        print("Best attribute "+ Best_Attribute + " -->")
        print(Best_Values)
        print(GainPos)
        Subs = GetSubs(RemainingData,Best_Attribute,Best_Values)
        #RemainingData = RemainingData.drop(columns=[Best_Attribute])
        #headers = list(RemainingData.columns.values)
        print(Subs)
        return [SplitAttributes(Subs[i],headers) for i in range(len(Subs))]

La función GetSubs recibe como entrada la data actual, el mejor atributo calculado de la función SplitAttribites y los valores asociados a ese atributo, y devuelve una colección de los dataframes asociados a esos valores.

In [14]:
def GetSubs(data,Best_Attribute, Best_Values):
    data_temp = data.copy()
    dataframe_collection = {} 
    for i in range(len(Best_Values)):
        dataframe_collection[i] = data_temp[data_temp[Best_Attribute] == Best_Values[i]].drop(columns= [Best_Attribute])
    return dataframe_collection

In [8]:
def CalculateNode(data):
    if len(data.unique()) == 1:
        return Node(data.unique()[0])
    # TODO, base case 2

In [9]:
data, X, y = SplitData("Golf_Other_Way.csv")
data_temp = data.copy()
headers = list(data_temp.columns.values)
print(headers)

['Temp', 'Outlook', 'Humidity', 'Play Golf']


In [22]:
Splitter = SplitAttributes(data_temp,headers)
Splitter

Current headers ['Temp', 'Outlook', 'Humidity', 'Play Golf']
Best attribute Outlook -->
['Rainy', 'Sunny', 'Overcast']
0.1181062751466107
{0:     Temp Humidity Play Golf
0    Hot   Normal        No
3    Hot   Normal       Yes
4   Cool   Normal       Yes
5   Cool     High        No
9   Cool   Normal       Yes
13   Hot     High       Yes, 1:     Temp Humidity Play Golf
1    Hot     High        No
7    Hot   Normal        No
8   Cool   Normal       Yes
10  Cool     High       Yes, 2:     Temp Humidity Play Golf
2    Hot   Normal       Yes
6   Cool     High       Yes
11   Hot     High       Yes
12  Cool   Normal       Yes}
Current headers ['Temp', 'Humidity', 'Play Golf']
Best attribute Humidity -->
['Normal', 'High']
0.04803508424256193
{0:    Temp Play Golf
0   Hot        No
3   Hot       Yes
4  Cool       Yes
9  Cool       Yes, 1:     Temp Play Golf
5   Cool        No
13   Hot       Yes}
Current headers ['Temp', 'Play Golf']
Best attribute Temp -->
['Cool', 'Hot']
0.31127812445913283
{0

[[[None, None], [None, None]],
 [[None, None], [None, None]],
 [[None, None], [None, None]]]

In [11]:
data_temp[data_temp['Outlook'] == 'Rainy'].drop(columns= ['Outlook'])

,Temp,Humidity,Play Golf
0,Hot,Normal,No
3,Hot,Normal,Yes
4,Cool,Normal,Yes
5,Cool,High,No
9,Cool,Normal,Yes
13,Hot,High,Yes


In [12]:
data_temp

,Temp,Outlook,Humidity,Play Golf
0,Hot,Rainy,Normal,No
1,Hot,Sunny,High,No
2,Hot,Overcast,Normal,Yes
3,Hot,Rainy,Normal,Yes
4,Cool,Rainy,Normal,Yes
5,Cool,Rainy,High,No
6,Cool,Overcast,High,Yes
7,Hot,Sunny,Normal,No
8,Cool,Sunny,Normal,Yes
9,Cool,Rainy,Normal,Yes


In [13]:
data

,Temp,Outlook,Humidity,Play Golf
0,Hot,Rainy,Normal,No
1,Hot,Sunny,High,No
2,Hot,Overcast,Normal,Yes
3,Hot,Rainy,Normal,Yes
4,Cool,Rainy,Normal,Yes
5,Cool,Rainy,High,No
6,Cool,Overcast,High,Yes
7,Hot,Sunny,Normal,No
8,Cool,Sunny,Normal,Yes
9,Cool,Rainy,Normal,Yes
